In [36]:
### import radd
from radd import build, CORE
from radd.tools import analyze
from itertools import takewhile, accumulate
from scipy.stats.mstats_extras import mjci
from copy import deepcopy
from numpy.random import random_sample as rs

homedir = os.path.expanduser('~')

ssb = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SSBayes/alldata_clean.csv"))
udf = ssb[ssb.cond.isin(['uniform'])]
edf = ssb[ssb.cond.isin(['early'])]
ldf = ssb[ssb.cond.isin(['late'])]

re = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SS/Reactive/Re_Data.csv"), index_col=0)
cox = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/Coxon/coxon_behav_data_binned.csv"))

inits = {'a': 0.45, 'v': 1.26, 'xb': 1.3, 'ssv': -1.3, 'tr': 0.2}

In [34]:
m.fitparams['y'] = m.observed[0]
m.fitparams['wts'] = m.cost_wts[0]
m.make_simulator(m.fitparams, p=inits)
sim = m.simulator

In [37]:
p=dict(deepcopy(inits))

p = sim.vectorize_params(p)
Pg, Tg = sim.__update_go_process__(p)
Ps, Ts = sim.__update_stop_process__(p)

In [39]:
ntot = sim.ntot
nss = sim.nss
nl = sim.nlevels
dx = sim.dx
#def recur():


In [84]:
%%timeit
np.where((np.random.random_sample((nl, Tg.max())).T < Pg), dx, -dx)

The slowest run took 9.36 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 18.1 µs per loop


In [88]:
lambda Tg, Pg,  = sim.xtb[:] * np.cumsum(np.where((rs((nl, Tg.max())).T < Pg), dx, -dx).T, axis=1)

In [92]:
%%timeit
sim.analyze_proactive(DVg, p)

100 loops, best of 3: 8.75 ms per loop


In [77]:
p['a']

array([ 0.44999999], dtype=float32)

In [78]:
%%timeit
np.cumsum(y[0][:340])[-1]

0.45014871522709887

In [93]:
if bool:
    print('k')

k


In [94]:
sum(takewhile(bool, (value < p['a'] for value in accumulate(y[i]))))

NameError: name 'i' is not defined

In [80]:
%%timeit
sum(takewhile(bool, (value < p['a'] for value in accumulate(y[i]))))




1000 loops, best of 3: 1.06 ms per loop


In [65]:
z = takewhile(lambda x: x<p['a'], x)

In [69]:
Tg

array([454, 454])

In [6]:
data = re.copy()

data['cond'] = 'flat'

In [29]:
m = build.Model(data = re, kind='xdpm', fit_on='average')#, depends_on={'v':'Cond'})

In [31]:
m.__prepare_fit__()

In [5]:
#params = np.sort(
x = m.inits.keys()

#).tolist()

In [15]:
x = list(m.inits)

In [16]:
x

['ssv', 'a', 'v', 'tr', 'xb']

In [23]:
x= m.observed[0]
#x.squeeze(axis=0).shape
#m.observed[0].shape

In [24]:
x

array([ 0.93463,  0.99385,  0.98402,  0.91107,  0.54959,  0.14221,
        0.51429,  0.54567,  0.56627,  0.58647,  0.61522,  0.49105,
        0.5308 ,  0.55047,  0.56965,  0.59579])

In [21]:
x.squeeze()

array([[ 0.94201,  0.99426,  0.98279,  0.89672,  0.50492,  0.10328,
         0.51263,  0.54257,  0.56297,  0.58212,  0.61031,  0.49795,
         0.52919,  0.54817,  0.56757,  0.59428],
       [ 0.92725,  0.99344,  0.98525,  0.92541,  0.59426,  0.18115,
         0.51652,  0.54902,  0.56989,  0.59031,  0.6177 ,  0.49691,
         0.52939,  0.55316,  0.57166,  0.59969]])

In [23]:
m.nlevels
m.observed[0][0].ndim

1

In [41]:
df=m.data
groups=['idx', 'Cond']
percentiles=np.array([.1, .3, .5, .7, .9])
mjcix = lambda x: mjci(x.rt, prob=percentiles)
# sort by ttype first to ensure go(acc==1) before stop(acc==0)


In [60]:
groups=['idx', 'Cond']
groups[1:]
if len(groups)>1:
        conds = groups[1:]
        nlevels = np.sum([df[c].unique().size for c in conds])
nidx = df.idx.unique().size
nquant = percentiles.size
nacc = 2


groups = np.hstack([groups, 'ttype', 'acc']).tolist()

In [61]:
godf = df.query('response==1')
godf_grouped = godf.groupby(groups)
categories = array([list(x) for x, xdf in godf_grouped])

qerr = np.vstack(godf_grouped.apply(mjcix).values)

In [68]:
catdf = pd.DataFrame(categories, columns=groups, index=np.arange(categories.shape[0]))

In [71]:
catdf.shape[0]

168

In [74]:
nidx * nlevels * nacc

168

In [76]:
quant_err = np.vstack(godf_grouped.apply(mjcix).values)
idx_qerr = quant_err.reshape(nidx, nlevels * nquant * nacc)
#np.nanmedian(qerr.reshape(nidx, nlevels * nquant * nacc), axis=1)
idx_medians = np.nanmedian(idx_qerr, axis=1)

In [78]:
idx_tiled_medians = [np.tile(idxmed, nlevels * nacc) for idxmed in idx_medians]
idx_tiled_medians = np.vstack(np.hstack(idx_tiled_medians))

In [82]:
quant_err.shape

(168, 5)

In [83]:
idx_qerr_ratio = idx_tiled_medians / quant_err

In [89]:
analyze.get_count_pwts(m.data, var='ssd', conds=m.conds)

TypeError: get_count_pwts() got an unexpected keyword argument 'conds'

In [86]:
analyze.mj_quanterr(df=m.data, groups=['idx', 'Cond'])

,idx,Cond,ttype,acc,10,30,50,70,90
0,1,pro,go,1,0.77681,1.45186,1.18323,0.85541,0.96141
1,1,pro,stop,0,0.64384,1.01316,1.12303,0.69940,0.30729
2,1,reac,go,1,2.33862,2.23801,1.87690,2.16512,1.11815
3,1,reac,stop,0,1.00337,0.74767,0.58437,0.99665,0.56937
4,2,pro,go,1,1.26713,1.06719,1.09538,1.31817,0.48936
5,2,pro,stop,0,0.84947,0.69641,0.94077,0.57967,1.37543
6,2,reac,go,1,1.49074,1.72682,2.32029,1.92553,1.79011
7,2,reac,stop,0,0.68143,0.80738,0.67626,0.34865,0.47449
8,3,pro,go,1,1.36421,1.36129,1.19636,1.35125,0.75967
9,3,pro,stop,0,0.01702,0.83062,1.13456,0.67270,0.39700


In [33]:
m.observedDF.groupby('Cond').mean()

,idx,acc,200,250,300,350,400,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
Cond,,,,,,,,,,,,,,,,,
bsl,60.60656,0.94201,0.99426,0.98279,0.89672,0.50492,0.10328,0.51263,0.54257,0.56297,0.58212,0.61031,0.49795,0.52919,0.54817,0.56757,0.59428
pnl,60.60656,0.92725,0.99344,0.98525,0.92541,0.59426,0.18115,0.51652,0.54902,0.56989,0.59031,0.61770,0.49691,0.52939,0.55316,0.57166,0.59969


In [8]:
m.observed

array([ 0.97521,  1.     ,  1.     ,  0.95   ,  0.8    ,  0.1    ,
        0.51767,  0.54517,  0.57184,  0.59735,  0.61186,  0.51983,
        0.54525,  0.55858,  0.57181,  0.60212])

In [ ]:
m.optimize(maxfev=5000, tol=1.e-4, btol=1.e-2, multiopt=True, nrand_inits=5, fit_flat=True, fit_cond=True, method='nelder', bdisp=True, disp=True)

In [540]:
ssdf.loc[0]

idx                1
Cond             pro
GoTrial            0
StopSignal    630.25
response       0.475
acc            0.525
rt            0.9019
ssd           630.25
Name: 0, dtype: object

In [563]:
nrt = (1-ssdf.acc)*stop_grouped.count().reset_index()['response']
get_ssrt = lambda df, idx: np.sort(df.rt.values)[int(nrt.loc[idx])] - ssdf.ssd.loc[idx]*.001
ssrt1 = [get_ssrt(df, xc[0]) for xc, df in cox[cox.ttype=='go'].groupby(['idx', 'Cond'])]


In [ ]:
ssrt

In [564]:
cox = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/Coxon/coxon_behav_data.csv"))
go_grouped = cox[(cox.ttype=='go')].groupby(['idx', 'Cond'])
stop_grouped = cox[cox.ttype=='stop'].groupby(['idx', 'Cond'])
ssdf = stop_grouped.mean().reset_index()#['ssd']
rtdf = go_grouped.median().reset_index()#['rt']
ssrt2 = (rtdf.rt - ssdf.ssd*.001)

In [562]:
rtdf.rt

0     0.8095
1     0.7960
2     0.8150
3     0.8070
4     0.8130
5     0.8160
6     0.8095
7     0.8020
8     0.8270
9     0.8300
10    0.8245
11    0.8190
12    0.8235
13    0.8135
14    0.8310
       ...  
69    0.8560
70    0.8220
71    0.8040
72    0.8485
73    0.8530
74    0.8120
75    0.8150
76    0.8350
77    0.8290
78    0.8380
79    0.8320
80    0.8260
81    0.8280
82    0.8050
83    0.8040
Name: rt, dtype: float64

In [565]:
from scipy.stats import pearsonr

In [566]:

pearsonr(ssrt1, ssrt2)

(0.038549481585749799, 0.72772955530980654)

In [67]:
data = m.handler.data
nperc = m.handler.percentiles.size
all_qp_cols = m.handler.all_qp_cols
idx_qp_cols = m.handler.idx_qp_cols

nrows = m.nidx * m.nlevels * m.nconds
ncols = len(all_qp_cols)

idx_conds = np.hstack(['idx', m.conds]).tolist()
ic_grp = m.data.groupby(idx_conds)
i_grp = m.data.groupby('idx')


In [ ]:
popt={'a': 0.5386462211608887, 'xb': 0.9047960042953491, 'ssv': -1.1062751962701585, 'tr': 0.1727830171585083, 'v': array([ 1.27869])}
p={'a': 0.5716851949691772, 'xb': 0.9122487306594849, 'ssv': -1.3582628022423542, 'tr': 0.11375404894351959, 'v': 1.16312}

In [ ]:
popt={'a': 0.53864622, 'v_pnl': 1.2505972481556207, 'v': array([ 1.29872,  1.2506 ]), 'xb': 0.904796, 'ssv': -1.1062751962701585, 'tr': 0.17278302, 'v_bsl': 1.2987241827070928}

In [295]:
m = build.Model(data=udf, depends_on={'v':'cond'}, kind='xdpm', fit_on='subjects')
#rem = build.Model(data=re, depends_on={'v':'Cond'}, kind='xdpm')

In [94]:
m.cost_wts

[array([[ 1.     ,  1.     ,  1.     ,  1.     ,  1.     ,  1.     ,
          5.33144,  2.8864 ,  2.28254,  4.46287,  1.0423 ]]),
 array([[ 1.     ,  1.     ,  1.     ,  1.     ,  1.     ,  1.     ,
          0.79499,  0.60444,  0.92873,  0.961  ,  0.39091]]),
 array([[  1.     ,   1.     ,   1.     ,   1.     ,   1.     ,   1.     ,
           1.99582,   2.5237 ,  82.52671,   5.21487,   0.63332]]),
 array([[ 1.     ,  1.     ,  1.     ,  1.     ,  1.     ,  1.     ,
          0.4186 ,  3.02798,  0.52339,  0.66713,  0.37978]]),
 array([[   1.     ,    1.     ,    1.07143,    1.     ,    0.9375 ,
            0.9375 ,    1.65207,    7.22586,    1.008  ,    0.99213,
          769.4184 ]]),
 array([[ 1.     ,  1.     ,  1.     ,  1.     ,  1.     ,  1.     ,
          0.37006,  0.7926 ,  0.92681,  1.0866 ,  0.40122]]),
 array([[  1.     ,   1.06667,   1.     ,   1.     ,   1.     ,   1.06667,
           2.37827,  57.15598,  16.82292,   1.47489,   1.03369]]),
 array([[ 1.     ,  1.     ,  

In [265]:
m = deepcopy(rem)
weight_presponse=True

In [165]:
idx_cols = ['idx', 'acc']
nrows = m.nidx

if True:
    conds = m.conds
    idx_cols.insert(1, m.conds)

In [166]:

idx_cols = np.hstack(idx_cols)

In [249]:
groups = 'idx'
#groups = ['idx', 'Cond']
get_vals = lambda df: df.dropna(axis=1).loc[:, 'acc':].values
group_df = lambda df: df.groupby(groups)

In [254]:
groups = 'idx'
#groups = ['idx', 'Cond']
grpData = m.data.groupby(groups)
#vals = grpd.apply(get_vals).values


In [ ]:
data = m.data
nperc = m.percentiles.size
all_qp_cols = m.handler.all_qp_cols
idx_qp_cols = m.handler.idx_qp_cols

ncols = len(all_qp_cols)

In [296]:

groups = 'idx'
nrows = m.nidx


# groups = ['idx', 'Cond']
# nrows = m.nidx * m.nlevels * m.nconds

grpData = m.data.groupby(groups)

datdf = grpData.apply(m.rangl_data).sortlevel(1)
dfvals = [np.hstack(datdf.values[i].astype(float)) for i in xrange(nrows)]

In [318]:
groups = ['idx', 'Cond']
#groups = 'idx'
#obsdf_cols = np.hstack([groups, 'acc'])

In [325]:
gdf = cox.groupby(['idx', 'Cond'])
x = gdf.apply(bin_idx_ssd).reset_index(drop=True)

In [361]:
from radd import CORE
CORE = reload(CORE)
build = reload(build)

In [382]:
ssd_list = np.asarray([1000*issd for issd in m.ssd])

In [384]:
ssd_list[0]

array([ 200.,  250.,  300.,  350.,  400.])

In [ ]:
np.asarray()

In [494]:
groups = ['idx', 'Cond']
grpData = cox.groupby(groups)
nrows = datdf.values.shape[0]

idxdf_cols, obsdf_cols, infocols = m.handler.__get_headers__()

ncols = len(obsdf_cols)
index = range(nrows)

In [495]:
datdf = grpData.apply(analyze.rangl_data).sortlevel(0)
#dfvals = [datdf.values[i].astype(float) for i in xrange(nrows)]
dfvals = [datdf.values[i].astype(float) for i in index]

In [489]:
idxdf_cols[rowi]

dfvals[rowi]

array([ 0.99174,  1.     ,  1.     ,  1.     ,  0.9    ,  0.     ,
        0.52504,  0.54513,  0.5585 ,  0.57196,  0.59844,  0.52676,
        0.5451 ,  0.55845,  0.57092,  0.59019])

In [458]:
dfvals[0].size

16

In [496]:

#idxcols, obsdf_cols, infocols = m.handler.__get_headers__()
observedDF = pd.DataFrame(np.zeros((nrows, ncols))*np.nan, columns=obsdf_cols, index=index)
for rowi in xrange(nrows):
    observedDF.loc[rowi, idxdf_cols[rowi]] = dfvals[rowi].tolist()

In [497]:
observedDF

,idx,Cond,acc,575,585,590,595,600,605,610,...,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,...,0.76784,0.79500,0.8095,0.82400,0.84916,0.76066,0.77700,0.7895,0.80356,0.82402
1,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.85714,NaN,...,0.76300,0.78300,0.7960,0.81634,0.84000,0.75642,0.76746,0.7850,0.79854,0.81216
2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.80000,NaN,...,0.78442,0.80200,0.8150,0.83500,0.85158,0.76300,0.78600,0.7970,0.81100,0.82200
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.83333,NaN,...,0.78200,0.79800,0.8070,0.81914,0.83600,0.76500,0.77900,0.7890,0.80722,0.82800
4,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,...,0.77282,0.79366,0.8130,0.83500,0.86200,0.75310,0.78300,0.7945,0.81754,0.84812
5,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.66667,...,0.77200,0.79600,0.8160,0.83500,0.86188,0.75272,0.76608,0.7860,0.80128,0.81456
6,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.66667,NaN,0.69231,...,0.77500,0.79600,0.8095,0.82700,0.84832,0.77304,0.78528,0.7980,0.80524,0.82996
7,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.80000,NaN,0.47059,...,0.76900,0.78800,0.8020,0.81600,0.84000,0.74354,0.77018,0.7865,0.79694,0.81790
8,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.78728,0.81200,0.8270,0.85268,0.87136,0.78252,0.80452,0.8190,0.83720,0.85776
9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.78800,0.81100,0.8300,0.84334,0.87018,0.77700,0.79556,0.8080,0.82324,0.87248


In [416]:
datdf.

idx  Cond
28   bsl     [0.99173553719, ...
     pnl     [0.97520661157, ...
29   bsl     [0.99173553719, ...
     pnl     [0.96694214876, ...
30   bsl     [0.942148760331,...
     pnl     [0.867768595041,...
31   bsl     [1.0, 1.0, 1.0, ...
     pnl     [0.96694214876, ...
32   bsl     [0.95041322314, ...
     pnl     [0.96694214876, ...
33   bsl     [0.942148760331,...
     pnl     [0.909090909091,...
34   bsl     [0.95041322314, ...
     pnl     [0.98347107438, ...
35   bsl     [0.96694214876, ...
                    ...         
85   pnl     [0.99173553719, ...
86   bsl     [0.933884297521,...
     pnl     [0.884297520661,...
87   bsl     [1.0, 1.0, 1.0, ...
     pnl     [1.0, 1.0, 1.0, ...
88   bsl     [0.809917355372,...
     pnl     [0.719008264463,...
89   bsl     [0.97520661157, ...
     pnl     [0.95867768595, ...
90   bsl     [0.99173553719, ...
     pnl     [0.933884297521,...
95   bsl     [1.0, 1.0, 0.9, ...
     pnl     [0.97520661157, ...
97   bsl     [1.0, 1.0, 1.0, ...


In [498]:
m = build.Model(data=cox, depends_on={'v':'Cond'}, kind='xdpm', fit_on='subjects')

['idx', 'Cond']


In [500]:
m.__make_dataframes__()

In [501]:
m.observed

[array([[ 1.     ,  1.     ,  0.76923,  0.57143,  0.54167,  0.16667,
          0.76784,  0.795  ,  0.8095 ,  0.824  ,  0.84916,  0.76066,
          0.777  ,  0.7895 ,  0.80356,  0.82402],
        [ 1.     ,  0.85714,  0.78947,  0.33333,  0.36364,  0.     ,
          0.763  ,  0.783  ,  0.796  ,  0.81634,  0.84   ,  0.75642,
          0.76746,  0.785  ,  0.79854,  0.81216]]),
 array([[ 1.     ,  0.8    ,  0.75   ,  0.55   ,  0.42105,  0.4    ,
          0.11111,  0.78442,  0.802  ,  0.815  ,  0.835  ,  0.85158,
          0.763  ,  0.786  ,  0.797  ,  0.811  ,  0.822  ],
        [ 1.     ,  0.83333,  0.66667,  0.5    ,  0.375  ,  0.125  ,
          0.     ,  0.782  ,  0.798  ,  0.807  ,  0.81914,  0.836  ,
          0.765  ,  0.779  ,  0.789  ,  0.80722,  0.828  ]]),
 array([[ 1.     ,  1.     ,  0.66667,  0.55556,  0.47368,  0.4    ,
          0.25   ,  0.77282,  0.79366,  0.813  ,  0.835  ,  0.862  ,
          0.7531 ,  0.783  ,  0.7945 ,  0.81754,  0.84812],
        [ 1.     ,  0.6666

In [ ]:
self.datdf = grpData.apply(self.model.rangl_data).sortlevel(0)
self.dfvals = [self.datdf.values[i].astype(float) for i in index]

In [446]:
i, o, info = m.handler.__get_headers__()

['idx',
 'Cond',
 'acc',
 605,
 615,
 625,
 635,
 645,
 605,
 615,
 625,
 635,
 645,
 605,
 615,
 625,
 635,
 645,
 655,
 605,
 615,
 625,
 635,
 645,
 655,
 610,
 620,
 630,
 640,
 650,
 660,
 610,
 620,
 630,
 640,
 650,
 660,
 600,
 610,
 620,
 630,
 640,
 600,
 610,
 620,
 630,
 640,
 620,
 630,
 640,
 650,
 660,
 620,
 630,
 640,
 650,
 660,
 630,
 640,
 650,
 660,
 670,
 630,
 640,
 650,
 660,
 670,
 615,
 625,
 635,
 645,
 655,
 665,
 615,
 625,
 635,
 645,
 655,
 665,
 600,
 610,
 620,
 630,
 640,
 650,
 600,
 610,
 620,
 630,
 640,
 650,
 610,
 620,
 630,
 640,
 650,
 610,
 620,
 630,
 640,
 650,
 640,
 650,
 660,
 670,
 680,
 620,
 630,
 640,
 650,
 660,
 585,
 595,
 605,
 615,
 625,
 585,
 595,
 605,
 615,
 625,
 600,
 610,
 620,
 630,
 640,
 600,
 610,
 620,
 630,
 640,
 615,
 625,
 635,
 645,
 655,
 665,
 675,
 615,
 625,
 635,
 645,
 655,
 665,
 675,
 600,
 610,
 620,
 630,
 600,
 610,
 620,
 630,
 610,
 620,
 630,
 640,
 650,
 610,
 620,
 630,
 640,
 650,
 610,
 620,
 63

In [396]:
m.observed

[array([[ 0.99174,  1.     ,  1.     ,  0.95   ,  0.6    ,  0.     ,
          0.50509,  0.53191,  0.55812,  0.56643,  0.59818,  0.49608,
          0.53166,  0.53187,  0.54585,  0.56744],
        [ 0.97521,  1.     ,  1.     ,  0.95   ,  0.8    ,  0.1    ,
          0.51767,  0.54517,  0.57184,  0.59735,  0.61186,  0.51983,
          0.54525,  0.55858,  0.57181,  0.60212]]),
 array([[ 0.99174,  1.     ,  1.     ,  1.     ,  0.9    ,  0.     ,
          0.52504,  0.54513,  0.5585 ,  0.57196,  0.59844,  0.52676,
          0.5451 ,  0.55845,  0.57092,  0.59019],
        [ 0.96694,  1.     ,  1.     ,  1.     ,  0.75   ,  0.35   ,
          0.53145,  0.55997,  0.58502,  0.59849,  0.62498,  0.53144,
          0.54477,  0.55171,  0.57507,  0.59804]]),
 array([[ 0.94215,  1.     ,  1.     ,  1.     ,  0.8    ,  0.25   ,
          0.52978,  0.57188,  0.59812,  0.61175,  0.63843,  0.53607,
          0.56057,  0.58535,  0.6098 ,  0.62076],
        [ 0.86777,  1.     ,  1.     ,  1.     ,  0.65  

In [310]:
obsdf_cols

array(['idx', 'Cond', 'acc'], 
      dtype='|S4')

In [153]:
dfvals = model.handler.dfvals
nlevels = model.nlevels
nconds = model.nconds
nsplits = nlevels * nconds
nidx = model.nidx
nrows = nidx * nconds * nlevels
nquant = model.percentiles.size
quant_cols = np.asarray(model.percentiles*100).astype(int)

# use martinz-jarett method to estimate quantile errs
quantdf = analyze.mj_quanterr(df=model.data, conds=model.conds, percentiles=model.percentiles)
idx_qwts = quantdf.loc[:, quant_cols].values.reshape(nidx*2, nsplits*nquant)

In [155]:
if weight_presponse:
    print(2)
    idx_pwts = analyze.get_count_pwts(model.data, var='ssd', conds=model.conds)
else:
    idx_pwts = [np.ones(idat.size - 2*nquant) for idat in dfvals]

cost_wts = np.array([np.append(pw, qw) for pw, qw in zip(idx_pwts, idx_qwts)])
cost_wts = [np.vstack(cost_wts[i:i+nsplits]) for i in range(0, nrows, nsplits)]
flat_cost_wts = [idx_cwts.mean(axis=0) for idx_cwts in cost_wts]

2


In [136]:
model.conds

['cond']

In [141]:
df = model.data

var='ssd'
conds=['cond']

if var=='ssd':
    df = df[df.ttype=='stop'].copy()


def calc_pwts(df_i):
    countdf = df_i.groupby(var).count()
    # get counts across levels except for last (correct Go//SSD==1000)
    lvl_counts = countdf.reset_index()['response'].values
    # calculate count ratio for all values
    varwts = np.median(lvl_counts)/lvl_counts
    # append 1 to front as weight of correct P(Go|No_SS)
    acc_var_wts = np.append(1., varwts)
    return acc_var_wts

ic_grp = df.groupby(np.hstack(['idx', conds]).tolist())
idx_pwts = [calc_pwts(cdf) for c, cdf in ic_grp]


#idx_var_wts = [np.median(cts)/cts for cts in counts]
# append 1 to front of each vector for weight of correct P(Go|No_SS)
#idx_var_pwts = [np.append(1., iwts) for iwts in idx_var_wts]

In [140]:
calc_pwts(cdf)

array([ 1.,  1.,  1.,  1.,  1.,  1.])

In [50]:
rem.observed[]

[array([ 0.94201,  0.99426,  0.98279,  0.89672,  0.50492,  0.10328,
         0.51263,  0.54257,  0.56297,  0.58212,  0.61031,  0.49795,
         0.52919,  0.54817,  0.56757,  0.59428,  0.92725,  0.99344,
         0.98525,  0.92541,  0.59426,  0.18115,  0.51652,  0.54902,
         0.56989,  0.59031,  0.6177 ,  0.49691,  0.52939,  0.55316,
         0.57166,  0.59969])]

In [51]:
plt.plot(m.observed[0])
#plt.plot(rem.observed[0])

In [15]:
godf = ssb[ssb.ttype=='go']#ssd.isin([0])]
ssb.replace({'ssd': {0: 1000}}, inplace=True)

In [9]:

ssb[ssb.ssd.isin(np.sort(ssb['ssd'].unique()))].uq


array(['stop'], dtype=object)